In [ ]:
import numpy as np
import cv2

m = cv2.imread('G:\\jorge\\Datasets\\PNOA\\chequeadas_mask\\_baleares-13199_crop3.png', cv2.IMREAD_GRAYSCALE)
print(np.unique(m))  

[  0   1   3   4   5   7   8   9  11  12  13  15  16  17  19  20  21  23
  24  25  27  28  29  31  32  33  35  36  37  39  40  41  43  44  45  47
  48  49  51  52  53  55  56  57  59  60  61  63  64  65  67  68  69  71
  72  73  75  76  77  79  80  81  83  84  85  87  88  89  91  92  93  95
  96  97  99 100 101 103 104 105 107 108 109 111 112 113 115 116 117 119
 120 121 123 124 125 127 128 129 131 132 133 135 136 137 139 140 141 143
 144 145 147 148 149 151 152 153 155 156 157 159 160 161 163 164 165 167
 168 169 171 172 173 175 176 177 179 180 181 183 184 185 187 188 189 191
 192 193 195 196 197 199 200 201 203 204 205 207 208 209 211 212 213 215
 216 217 219 220 221 223 224 225 227 228 229 231 232 233 235 236 237 239
 240 241 243 244 245 247 248 249 251 252 253 255]


In [ ]:
import os
import cv2
import numpy as np

input_dir  = 'G:\\jorge\\Datasets\\mascaras_originales' 
output_dir = 'G:\\jorge\\Datasets\\mascaras_binarias_png'
os.makedirs(output_dir, exist_ok=True)

THRESH = 128

for fname in os.listdir(input_dir):
    if not (fname.lower().endswith('.tif') or fname.lower().endswith('.png')):
        continue

    path_in = os.path.join(input_dir, fname)
    mask = cv2.imread(path_in, cv2.IMREAD_UNCHANGED)
    if mask is None:
        print(f"¡Error leyendo {fname}, comprueba que existe y es una imagen válida!")
        continue

    if mask.ndim == 3:
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

    mask_bin = (mask > THRESH).astype(np.uint8)

    name_out = os.path.splitext(fname)[0] + '.png'
    path_out = os.path.join(output_dir, name_out)
    cv2.imwrite(path_out, mask_bin)


In [ ]:
import os
from PIL import Image
import numpy as np
import cv2

input_dir  = 'G:\\jorge\\Datasets\\images_512x512\\train\\gt'         
output_dir = 'G:\\jorge\\Datasets\\mascaras_binarias_png'    
os.makedirs(output_dir, exist_ok=True)

for fname in os.listdir(input_dir):
    if not fname.lower().endswith(('.tif', '.tiff')):
        continue

    path_in = os.path.join(input_dir, fname)
    im = Image.open(path_in).convert('L')  
    arr = np.array(im)

    mask_bin = (arr > 128).astype(np.uint8)

    out_name = os.path.splitext(fname)[0] + '.png'
    path_out = os.path.join(output_dir, out_name)
    Image.fromarray(mask_bin, mode='L').save(path_out)

    # 4) Verifica leyendo con OpenCV
    reim = cv2.imread(path_out, cv2.IMREAD_GRAYSCALE)
    if reim is None:
        print(f"No se pudo leer de nuevo {path_out}")
    else:
        print(f"{out_name}: valores únicos → {np.unique(reim)}")


In [ ]:
print("Conversión completada. Verifica con:")
import cv2, numpy as np
m = cv2.imread('G:\\jorge\\Datasets\\mascaras_binarias_png\\vienna9_pos_994_994_crop4.png', cv2.IMREAD_GRAYSCALE)
print(np.unique(m))  

Conversión completada. Verifica con:
[0 1]


In [ ]:
import os
from PIL import Image
import numpy as np
import shutil

tif_dir     = 'G:\\jorge\\Datasets\\images_512x512\\train\\images'                    
png_dest    = 'G:\\jorge\\Datasets\\INRIA-PNOA\\images'
os.makedirs(png_dest, exist_ok=True)

for fname in os.listdir(tif_dir):
    if not fname.lower().endswith(('.tif', '.tiff')):
        continue

    path_in  = os.path.join(tif_dir, fname)
    name_png = os.path.splitext(fname)[0] + '.png'
    path_out = os.path.join(png_dest,   name_png)

    with Image.open(path_in) as im:
        im_rgb = im.convert('RGB')                 
        arr    = np.array(im_rgb)

        if arr.dtype != np.uint8:
            arr = ((arr - arr.min()) / (arr.max() - arr.min()) * 255).astype(np.uint8)
            im_rgb = Image.fromarray(arr, 'RGB')

        im_rgb.save(path_out, format='PNG')

    print(f'Convertido y movido: {name_png}')

print('¡Conversión completa! Ahora tienes todos los .png junto con el resto de imágenes.')


In [ ]:
import os
import random
import shutil

src_images = 'G:\\jorge\\Datasets\\INRIA-PNOA\\images'   
src_masks  = 'G:\\jorge\\Datasets\\INRIA-PNOA\\mascaras_binarias_png'   
dst_base   = 'G:\\jorge\\Datasets\\INRIA-PNOA'     

frac_train = 0.70
frac_val   = 0.15

for split in ['train','val','test']:
    for kind in ['images','masks']:
        d = os.path.join(dst_base, split, kind)
        os.makedirs(d, exist_ok=True)

files = [os.path.splitext(f)[0]
         for f in os.listdir(src_images)
         if f.lower().endswith('.png')]
files.sort()
random.seed(42)
random.shuffle(files)

n = len(files)
n_train = int(n * frac_train)
n_val   = int(n * frac_val)

splits = {
    'train': files[:n_train],
    'val':   files[n_train:n_train + n_val],
    'test':  files[n_train + n_val:]
}

for split, names in splits.items():
    for name in names:
        img_src  = os.path.join(src_images, name + '.png')
        msk_src  = os.path.join(src_masks,  name + '.png')
        img_dst  = os.path.join(dst_base, split, 'images', name + '.png')
        msk_dst  = os.path.join(dst_base, split, 'masks',  name + '.png')
        if not os.path.isfile(img_src):
            raise FileNotFoundError(f"No encuentro imagen {img_src}")
        if not os.path.isfile(msk_src):
            raise FileNotFoundError(f"No encuentro máscara {msk_src}")
        shutil.copy(img_src, img_dst)
        shutil.copy(msk_src, msk_dst)

print("Reparto completado:")
for split in splits:
    print(f"  {split:5s}: {len(splits[split])} pares (imagen+máscara)")


Reparto completado:
  train: 19479 pares (imagen+máscara)
  val  : 4174 pares (imagen+máscara)
  test : 4175 pares (imagen+máscara)
